In [ ]:
from pytube import YouTube
import ffmpeg
import logging
import subprocess
from pathlib import Path
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [ ]:
text = "https://www.youtube.com/watch?v=w4vrOUau2iY"
yt = YouTube(text)

stream_url = yt.streams.all()[0].url  # Get the URL of the video stream
title = yt.title.replace(" ", "_").replace("/", "_").replace("-", "")
# filename = f"{title}.wav".lower()
filename = f"test.wav".lower()

whisper_model_path = Path("models/ggml-base.en.bin")
whisper_executable_path = Path("/Users/award40/Desktop/personal/github/generative-ai-workbook/learning/whisper/whisper.cpp/main")


test_wav_file = "/Users/award40/Desktop/personal/github/generative-ai-workbook/learning/whisper/test.wav"
"/Users/award40/Desktop/personal/github/generative-ai-workbook/learning/whisper/test_fixed.wav"


In [ ]:
def format_wav(path: Path, wav_path: Path) -> int:
    # audio, err = (
    #     ffmpeg
    #     .input(stream_url)
    #     .output("pipe:", format='wav', ar='16000', ac=1, acodec='pcm_s16le')  # Select WAV output format, and pcm_s16le auidio codec. My add ar=sample_rate
    #     .run(capture_stdout=True)
    # )

    # # Write the audio buffer to file for testing
    # with open(filename, 'wb') as f:
    #     f.write(audio)

    command = f'ffmpeg -i "{path.absolute()}" -ar 16000 -ac 1 -c:a pcm_s16le "{wav_path.absolute()}" -y'
    p = subprocess.Popen(
        command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT
    )
    out, err = p.communicate()
    out_str: str = out.decode("utf-8")
    err_str: str = err.decode("utf-8")
    logger.info(f"FFMPEG output: {out_str}")
    logger.info(f"FFMPEG error: {err_str}")
    logger.info(f"FFMPEG return code: {p.returncode}")
    return p.returncode

In [ ]:
def wav_to_srt(wav_path: Path, srt_path: Path, whisper_executable_path: Path) -> int:
    if srt_path.exists():
        logger.info(f"File {srt_path} already exists, skipping")
        return 0

    command = [
        whisper_executable_path.as_posix(),
        f"{wav_path}",
        "-t",
        "4",  # 4 threads
        "--max-len",  # max length in characters
        "16",
        "-l",
        "auto",  # language auto
        "-m",  # model
        f"{whisper_model_path.as_posix()}",
        "-osrt",  # output srt
        "-of",  # output file
        f"{srt_path.with_suffix('').as_posix()}",  # weird choice in whisper.cpp it has to be without extension
    ]

    logger.info(f"Running command: {command}")
    p2 = subprocess.Popen(
        command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT
    )

    while p2.poll() is None:
        output = p2.stdout.readline()  # type: ignore
        logger.info(output)
    out, err = p2.communicate()
    out_str = out.decode("utf-8")
    err_str = err.decode("utf-8")

    logger.info(f"Transcription output: {out_str}")
    logger.info(f"Transcription error: {err_str}")
    logger.info(f"Transcription return code: {p2.returncode}")
    return p2.returncode